<a href="https://colab.research.google.com/github/shreejitachauhan/Data_engineering/blob/load/load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install snowflake-connector-python
!pip install "snowflake-connector-python[pandas]"

import snowflake.connector
import getpass

password = getpass.getpass("Enter Snowflake password: ")
conn = snowflake.connector.connect(
    user="SHREEJITACHAUHAN",
    password=password,
    account="FHTJQHV-WW96608",
    role="ACCOUNTADMIN",
    warehouse="COMPUTE_WH",
    database="DATASET",
    schema="SALES"
)
cursor = conn.cursor()

cursor.execute("CREATE OR REPLACE SCHEMA sales_dwh;")

cursor.execute("""
CREATE OR REPLACE TABLE sales_dwh.Dim_Date (
    Date_ID STRING PRIMARY KEY,
    DAY INT, MONTH INT, YEAR INT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP
);
""")
cursor.execute("""
CREATE OR REPLACE TABLE sales_dwh.Dim_Branch (
    Branch_ID STRING PRIMARY KEY,
    BRANCH_NAME STRING,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP
);
""")
cursor.execute("""
CREATE OR REPLACE TABLE sales_dwh.Dim_Dealer (
    Dealer_ID STRING PRIMARY KEY,
    DEALER_NAME STRING,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP
);
""")
cursor.execute("""
CREATE OR REPLACE TABLE sales_dwh.Dim_Product (
    Model_ID STRING PRIMARY KEY,
    PRODUCT_NAME STRING,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP
);
""")
cursor.execute("""
CREATE OR REPLACE TABLE sales_dwh.Fact_Sales (
    Date_ID STRING,
    Branch_ID STRING,
    Dealer_ID STRING,
    Model_ID STRING,
    Revenue INT,
    Units_Sold INT,
    Created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (Date_ID) REFERENCES sales_dwh.Dim_Date(Date_ID),
    FOREIGN KEY (Branch_ID) REFERENCES sales_dwh.Dim_Branch(Branch_ID),
    FOREIGN KEY (Dealer_ID) REFERENCES sales_dwh.Dim_Dealer(Dealer_ID),
    FOREIGN KEY (Model_ID) REFERENCES sales_dwh.Dim_Product(Model_ID)
);
""")

cursor.execute("""
INSERT INTO sales_dwh.Dim_Date (Date_ID, DAY, MONTH, YEAR)
SELECT Date_ID, DAY, MONTH, YEAR
FROM sales_temp.Dim_Date
WHERE Date_ID NOT IN (SELECT Date_ID FROM sales_dwh.Dim_Date);
""")
cursor.execute("""
INSERT INTO sales_dwh.Dim_Branch (Branch_ID, BRANCH_NAME)
SELECT Branch_ID, BRANCH_NAME
FROM sales_temp.Dim_Branch
WHERE Branch_ID NOT IN (SELECT Branch_ID FROM sales_dwh.Dim_Branch);
""")
cursor.execute("""
INSERT INTO sales_dwh.Dim_Dealer (Dealer_ID, DEALER_NAME)
SELECT Dealer_ID, DEALER_NAME
FROM sales_temp.Dim_Dealer
WHERE Dealer_ID NOT IN (SELECT Dealer_ID FROM sales_dwh.Dim_Dealer);
""")
cursor.execute("""
INSERT INTO sales_dwh.Dim_Product (Model_ID, PRODUCT_NAME)
SELECT Model_ID, PRODUCT_NAME
FROM sales_temp.Dim_Product
WHERE Model_ID NOT IN (SELECT Model_ID FROM sales_dwh.Dim_Product);
""")

cursor.execute("""
INSERT INTO sales_dwh.Fact_Sales (
    Date_ID, Branch_ID, Dealer_ID, Model_ID, Revenue, Units_Sold
)
SELECT f.Date_ID, f.Branch_ID, f.Dealer_ID, f.Model_ID, f.Revenue, f.Units_Sold
FROM sales_temp.Fact_Sales f
JOIN sales_dwh.Dim_Date d ON f.Date_ID = d.Date_ID
JOIN sales_dwh.Dim_Branch b ON f.Branch_ID = b.Branch_ID
JOIN sales_dwh.Dim_Dealer dl ON f.Dealer_ID = dl.Dealer_ID
JOIN sales_dwh.Dim_Product p ON f.Model_ID = p.Model_ID;
""")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 6.7 MB/s eta 0:00:00
Enter Snowflake password: ··········
